In [4]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
print('Libraries loaded')

Libraries loaded


In [5]:
words = open('names.txt','r').read().splitlines()
print('First five names:', words[:5])

First five names: ['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [6]:
# mappings of letters in the alphabet (a <-> 1)
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print('Mapping between intergers and letters:\n', itos)

Mapping between intergers and letters:
 {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [10]:
# build training, development and test data sets

def build_dataset(words):

    block_size = 3 # context length: how many characters do we take to predict the next one?
    X, Y = [], []

    count = 0
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append
            count += 1

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    return(X, Y)

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])
print('Size of Training X and Y    :', Xtr.shape, Ytr.shape)
print('Size of Development X and Y :', Xdev.shape, Ydev.shape)
print('Size of Test X and Y        :', Xte.shape, Yte.shape)

Size of Training X and Y    : torch.Size([182441, 3]) torch.Size([182441])
Size of Development X and Y : torch.Size([22902, 3]) torch.Size([22902])
Size of Test X and Y        : torch.Size([22803, 3]) torch.Size([22803])


In [42]:
# MLP
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27,10), generator=g)
W1 = torch.randn((30,200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = (C, W1, b1, W2, b2)
for p in parameters:
    p.requires_grad = True
print('Number of parameters in the MLP:', sum(p.nelement() for p in parameters))

Number of parameters in the MLP: 11897


In [48]:
# Train on training set
for i in range(10000):
    
    #construct minibatch
    batch_size = 32
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    
    # forward pass
    emb = C[Xtr[ix]] 
    XDim0 = batch_size
    h = torch.zeros((XDim0, 200))
    for j in range(XDim0):
        h[j] = torch.tanh(emb.view(-1, 30)[j,:] @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    for p in parameters:
        p.data += -0.1 * p.grad
        
print(loss.item())

2.5127789974212646


In [50]:
emb = C[Xtr] 
XDim0 = emb.shape[0]
h = torch.zeros((XDim0, 200))
for j in range(XDim0):
    h[j] = torch.tanh(emb.view(-1, 30)[j,:] @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
print(loss.item())

2.340430974960327


In [51]:
emb = C[Xdev] 
XDim0 = emb.shape[0]
h = torch.zeros((XDim0, 200))
for j in range(XDim0):
    h[j] = torch.tanh(emb.view(-1, 30)[j,:] @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
print(loss.item())

2.3719072341918945


In [52]:
# plt.figure(figsize=(8,8))
# plt.scatter(C[:,0].data, C[:,1].data, s=200)
# for i in range(C.shape[0]):
#     plt.text(C[i,0].item(), C[i,1].item(), itos[i], ha="center", va="center", color='white')
# plt.grid('minor')

In [53]:
# sample from the model
g = torch.Generator().manual_seed(2147483647)
block_size = 3

for p in range(20):
    
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
        
        print(''.join(itos[i] for i in out))

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0